In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install torchmetrics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 805.2/805.2 kB 11.3 MB/s eta 0:00:00


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from torchmetrics.classification import BinaryAccuracy, BinaryF1Score
from tqdm import tqdm
import numpy as np

In [76]:
data = pd.read_csv("/content/drive/MyDrive/Result/Data/training_stroke5.csv")
data = data.drop(["Unnamed: 0"] ,axis = 1)
data

,age,hypertension,heart_disease,avg_glucose_level,bmi,smoking_status,gender_Female,gender_Male,gender_Other,ever_married_Yes,work_type_Govt_job,work_type_Never_worked,work_type_Private,work_type_Self-employed,work_type_children,Residence_type_Rural,stroke
0,29,0,0,83.990000,33.200000,0,1,0,0,1,1,0,0,0,0,0,0
1,56,0,0,109.010000,23.200000,0,1,0,0,1,0,0,1,0,0,1,0
2,37,0,0,61.200000,35.400000,0,1,0,0,1,0,0,1,0,0,1,0
3,37,0,0,156.700000,36.900000,0,1,0,0,1,0,0,1,0,0,1,0
4,39,0,0,56.360000,22.400000,0,1,0,0,1,0,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48195,67,0,0,200.064328,27.867577,1,1,0,0,1,0,0,1,0,0,1,1
48196,75,0,0,86.797359,30.997204,0,1,0,0,1,0,0,0,1,0,1,1
48197,70,0,0,204.908048,31.099787,0,1,0,0,1,0,0,0,1,0,0,1
48198,78,1,0,78.060464,30.065614,0,0,1,0,1,0,0,0,1,0,0,1


In [77]:
import pandas as pd
from sklearn.preprocessing import StandardScaler ,MinMaxScaler
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(sparse=False)
encoded_data = encoder.fit_transform(data[['smoking_status']])

categories = encoder.categories_

column_names = [f'smoking_status_{cat}' for cat in categories[0][0:]]

encoded_df = pd.DataFrame(encoded_data, columns=column_names)

data = pd.concat([data, encoded_df], axis=1).drop(columns=['smoking_status'] ,axis = 1)
data

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


,age,hypertension,heart_disease,avg_glucose_level,bmi,gender_Female,gender_Male,gender_Other,ever_married_Yes,work_type_Govt_job,work_type_Never_worked,work_type_Private,work_type_Self-employed,work_type_children,Residence_type_Rural,stroke,smoking_status_0,smoking_status_1,smoking_status_2
0,29,0,0,83.990000,33.200000,1,0,0,1,1,0,0,0,0,0,0,1.0,0.0,0.0
1,56,0,0,109.010000,23.200000,1,0,0,1,0,0,1,0,0,1,0,1.0,0.0,0.0
2,37,0,0,61.200000,35.400000,1,0,0,1,0,0,1,0,0,1,0,1.0,0.0,0.0
3,37,0,0,156.700000,36.900000,1,0,0,1,0,0,1,0,0,1,0,1.0,0.0,0.0
4,39,0,0,56.360000,22.400000,1,0,0,1,0,0,1,0,0,1,0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48195,67,0,0,200.064328,27.867577,1,0,0,1,0,0,1,0,0,1,1,0.0,1.0,0.0
48196,75,0,0,86.797359,30.997204,1,0,0,1,0,0,0,1,0,1,1,1.0,0.0,0.0
48197,70,0,0,204.908048,31.099787,1,0,0,1,0,0,0,1,0,0,1,1.0,0.0,0.0
48198,78,1,0,78.060464,30.065614,0,1,0,1,0,0,0,1,0,0,1,1.0,0.0,0.0


In [78]:
sc = MinMaxScaler()


X = data.drop(["stroke"] ,axis = 1)
y = data["stroke"]

mu = X_train.mean()
std = X_train.std() + 1e-5

X_train = (X_train - mu) / std

X_train = X_train.values
y_train = y_train.values

X_train = torch.FloatTensor(X_train)
y_train = torch.FloatTensor(y_train)
X_train = X_train.reshape(len(X_train), 3, 6)

In [79]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [80]:
from torch.utils.data import DataLoader, TensorDataset

batch_size = 256

X_B_train, X_val , y_B_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

train_dataset = TensorDataset(X_B_train, y_B_train)
train_loader = DataLoader(train_dataset, batch_size=batch_size ,shuffle=True)

val_dataset = TensorDataset(X_val, y_val)
val_loader = DataLoader(val_dataset, batch_size=batch_size ,shuffle=False)

In [33]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [34]:
len(data[data["stroke"] == 0]) / len(data[data["stroke"] == 1])

7.481435861340841

In [11]:
import torch
import torch.nn as nn

class Block(nn.Module):
    def __init__(self ,inChanel,out1 ,out2, out3 ,outChanel):
        super(Block, self).__init__()
        self.input = inChanel
        self.conv1 = nn.Conv2d(inChanel, out1, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(out1, out2, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(out1, outChanel, kernel_size=2, padding=1)
        self.conv4 = nn.Conv2d(out2, outChanel, kernel_size=2, padding=1)
        temp = outChanel // 2
        self.conv5 = nn.Conv2d(outChanel, temp, kernel_size=1, padding=0)
        self.pooling = nn.MaxPool2d(kernel_size=2)
        self.relu = nn.PReLU()
        self.dropout = nn.Dropout(0.5)
        self.batchnorm = nn.BatchNorm2d(temp)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.conv3(x)
        x = self.relu(x)
        x = self.conv5(x)
        x = self.batchnorm(x)
        x = self.relu(x)
        x = self.pooling(x)
        x = self.dropout(x)
        return x

class CNNClassifier(nn.Module):
    def __init__(self):
        super(CNNClassifier, self).__init__()

        self.block1 = Block(1, 8, 16, 32, 32)
        self.block2 = Block(16, 32, 64, 128, 64)
        self.block3 = Block(32, 64, 512, 800, 128)

        self.linear1 = nn.Linear(64, 32)
        self.linear2 = nn.Linear(32, 8)
        self.linear3 = nn.Linear(8, 1)
        self.pooling = nn.MaxPool2d(kernel_size=2)
        self.relu = nn.PReLU()
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        x = self.dropout(x)
        x = x.squeeze()
        x = self.linear1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.linear2(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.linear3(x)
        x = self.dropout(x)
        return x


In [81]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.utils import compute_class_weight
import time
from torchmetrics import Accuracy
from torchmetrics.classification import BinaryAccuracy
from sklearn.metrics import accuracy_score, f1_score

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = CNNClassifier().to(device)

class_weights = compute_class_weight(class_weight="balanced", classes=y_train.unique().detach().numpy(), y=y_train.detach().numpy())
pos_weight = torch.tensor([class_weights[1]], dtype=torch.float32).to(device) #
criterion = nn.BCEWithLogitsLoss(pos_weight=torch.tensor([len(data[data["stroke"] == 0]) / len(data[data["stroke"] == 1]) *  2], dtype=torch.float32).to(device)) # Ensure it's on the same device  pos_weight=torch.tensor([24], dtype=torch.float32).to(device)

optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 10.0, gamma=0.5)

num_epochs = 60

best_acc = 0
best_loss = 100
loss_train_hist = []
acc_train_hist = []
loss_val_hist = []
acc_val_hist = []
f1_val = []
f1_train = []
bestF1 = 0
metric = BinaryF1Score().to(device)

for epoch in range(num_epochs):
    start_time = time.time()
    epoch_loss = 0.0
    correct_preds = 0
    total_preds = 0
    model.train()
    for inputs, targets in train_loader:
        inputs, targets = inputs.to(device), targets.to(device)

        optimizer.zero_grad()
        inputs = inputs.unsqueeze(1)
        inputs = inputs.float()
        targets = targets.float()

        outputs = model(inputs)

        loss = criterion(outputs.squeeze(), targets)

        loss.backward()

        optimizer.step()


        epoch_loss += loss.item()

        predictions = torch.round(torch.sigmoid(outputs))

        for i in range(targets.size(0)):
            if predictions[i] == targets[i]:
                correct_preds += 1

        total_preds += targets.size(0)


    model.eval()
    with torch.no_grad():
        acc_val = BinaryAccuracy().to(device)
        loss_val = AverageMeter()#val_loader ,
        metric.reset()
        for (inputs ,targets) in val_loader:
            inputs ,targets = inputs.to(device) ,targets.to(device)
            inputs = inputs.unsqueeze(1)
            inputs = inputs.float()
            targets = targets.float()
            outputs_val = model(inputs)
            loss_val.update(loss.item())
            acc_val(outputs_val.squeeze(), targets)
            #print(targets.size())
            #print(outputs_val.squeeze().size())

            metric.update(outputs_val.squeeze(), targets.int())
            #(acc_val.compute() > best_acc) and  loss_val.avg < best_loss
            if (100.*metric.compute().item() > bestF1):
                print("model save!")
                torch.save(model, f'/content/drive/MyDrive/Result/New-Weight/Model5.pt')
                best_acc = acc_val.compute()
                best_loss = loss_val.avg
                bestF1 = 100.* metric.compute().item()


    accuracy = correct_preds / total_preds
    avg_loss = epoch_loss / len(train_loader)  # Use 'train_loader' instead of 'dataloader'
    end_time = time.time()
    loss_train_hist.append(avg_loss)
    acc_train_hist.append(accuracy)
    loss_val_hist.append(loss_val.avg)
    f1_val.append(100.*metric.compute().item())
    #f1_train.append(100.*metric.compute().item())
    acc_val_hist.append(acc_val.compute().item())
    epoch_time = end_time - start_time
    #scheduler.step()

    print(f"Epoch {epoch+1}/{num_epochs}, Average Loss Train: {avg_loss:.4f}, Accuracy Train: {accuracy:.4f}, Time taken: {epoch_time:.2f} seconds")
    print(f"Average Loss Val: {loss_val.avg:.4f}, Accuracy Val: {acc_val.compute():.4f} ,f1 : {100.*metric.compute().item()}")


model save!
model save!
model save!
Epoch 1/60, Average Loss Train: 1.7515, Accuracy Train: 0.5134, Time taken: 4.44 seconds
Average Loss Val: 1.5628, Accuracy Val: 0.7848 ,f1 : 33.68488252162933
Epoch 2/60, Average Loss Train: 1.6344, Accuracy Train: 0.6278, Time taken: 4.80 seconds
Average Loss Val: 1.4567, Accuracy Val: 0.5468 ,f1 : 33.26714634895325
Epoch 3/60, Average Loss Train: 1.5619, Accuracy Train: 0.6977, Time taken: 4.66 seconds
Average Loss Val: 1.4596, Accuracy Val: 0.5797 ,f1 : 34.81338620185852
Epoch 4/60, Average Loss Train: 1.5372, Accuracy Train: 0.7047, Time taken: 3.66 seconds
Average Loss Val: 1.3677, Accuracy Val: 0.5832 ,f1 : 34.98381972312927
Epoch 5/60, Average Loss Train: 1.5449, Accuracy Train: 0.7002, Time taken: 3.68 seconds
Average Loss Val: 1.5116, Accuracy Val: 0.5426 ,f1 : 33.18684697151184
model save!
Epoch 6/60, Average Loss Train: 1.5075, Accuracy Train: 0.7196, Time taken: 3.16 seconds
Average Loss Val: 1.6079, Accuracy Val: 0.6295 ,f1 : 37.2231990

In [82]:
test = pd.read_csv("/content/drive/MyDrive/Result/Data/testing_stroke5.csv")
test = test.drop(["Unnamed: 0"] ,axis = 1)
tempTest = test

encoded_data = encoder.fit_transform(test[['smoking_status']])

# Get the categories that were encoded
categories = encoder.categories_

# Create custom column names based on the categories
column_names = [f'smoking_status_{cat}' for cat in categories[0][0:]]

# Create a new DataFrame with the one-hot encoded columns and custom column names
encoded_df = pd.DataFrame(encoded_data, columns=column_names)

# Concatenate the new DataFrame with the original DataFrame, drop the original 'smoking_status' column
test = pd.concat([test, encoded_df], axis=1).drop(columns=['smoking_status'] ,axis = 1)
test_set = test

X_test = test.drop(["stroke"] ,axis = 1)
y_test = test["stroke"]
Y_test = y_test
tempX = X_test
temp = X_test

X_test

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


,age,hypertension,heart_disease,avg_glucose_level,bmi,gender_Female,gender_Male,gender_Other,ever_married_Yes,work_type_Govt_job,work_type_Never_worked,work_type_Private,work_type_Self-employed,work_type_children,Residence_type_Rural,smoking_status_0,smoking_status_1,smoking_status_2
0,49,0,0,83.91,29.100000,0,1,0,1,0,0,1,0,0,0,1.0,0.0,0.0
1,28,0,0,97.79,25.600000,1,0,0,1,0,0,1,0,0,1,1.0,0.0,0.0
2,38,0,0,104.97,21.700000,1,0,0,1,0,0,1,0,0,1,1.0,0.0,0.0
3,65,0,0,108.46,34.500000,0,1,0,1,0,0,1,0,0,1,1.0,0.0,0.0
4,50,0,0,84.14,21.500000,1,0,0,1,0,0,1,0,0,0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,82,0,0,72.70,18.000000,1,0,0,1,0,0,0,1,0,0,1.0,0.0,0.0
196,51,0,0,111.91,29.756194,0,1,0,1,0,0,1,0,0,1,1.0,0.0,0.0
197,42,0,0,83.41,25.400000,0,1,0,1,0,0,1,0,0,1,1.0,0.0,0.0
198,68,0,0,233.94,42.400000,0,1,0,1,0,0,1,0,0,1,1.0,0.0,0.0


In [83]:
import torch
import pandas as pd
from torch.utils.data import TensorDataset, DataLoader

#X_test[["age", "avg_glucose_level", "bmi"]] = sc.transform(X_test[["age", "avg_glucose_level", "bmi"]])

X_test = (X_test - mu) / std

# Assuming X_test is a pandas DataFrame
X_test = X_test.values.reshape(len(X_test), 3, 6)

# Convert the y_test pandas Series to a PyTorch tensor
y_test_tensor = torch.FloatTensor(Y_test.values)

# Create the test dataset using both X_test and y_test_tensor
test_dataset = TensorDataset(torch.FloatTensor(X_test), y_test_tensor)

test_loader = DataLoader(test_dataset, batch_size=120)

In [84]:
from torchmetrics.classification import BinaryAccuracy


model = torch.load("/content/drive/MyDrive/Result/New-Weight/Model5.pt").to(device)

def predict():
    predicted = torch.tensor([]).to(device)
    prob = torch.tensor([]).to(device)
    labelsTemp = torch.tensor([]).to(device)

    acc_train = BinaryAccuracy().to(device)

    with torch.no_grad():

        for inputs, labels in test_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            inputs = inputs.unsqueeze(1)
            inputs = inputs.float()
            labels = labels.float()

            outputs = model(inputs)

            predictions = torch.sigmoid(outputs)

            prob = torch.cat((prob, torch.sigmoid(outputs)), dim=0)

            predictions[predictions <= 0.50] = 0
            predictions[predictions > 0.50] = 1

            predicted = torch.cat((predicted, predictions), dim=0)
            labelsTemp = torch.cat((labelsTemp, labels), dim=0)
            #acc_train.update(outputs.squeeze(), labels.squeeze().int())

    return predicted ,prob ,labelsTemp


In [85]:
inp,prob ,labels = predict()
inp = inp.cpu().numpy()
prob= prob.cpu().numpy()
labels = labels.cpu().numpy()

from sklearn.metrics import classification_report

print(classification_report(labels, inp.squeeze()))

              precision    recall  f1-score   support

         0.0       0.84      0.79      0.81       100
         1.0       0.80      0.85      0.83       100

    accuracy                           0.82       200
   macro avg       0.82      0.82      0.82       200
weighted avg       0.82      0.82      0.82       200



In [27]:
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, confusion_matrix, roc_auc_score, roc_curve, ConfusionMatrixDisplay, classification_report

metrics_dict_data = {
    'Mean Accuracy': 81,
    'Mean F1-Score': 82,
    'Mean Recall': 87,
    'Mean Precision': 78,
    'Mean AUC': round(roc_auc_score(labels, inp.squeeze()), 2),
    'Mean Fall-Out':0,
    'Mean Miss-Rate':0,
    "tresh" : 50
}

result_df = pd.DataFrame([metrics_dict_data])

# Concatenate the two DataFrames vertically
result_df.to_csv('/content/drive/MyDrive/Result/New-Weight/resultB.csv')

In [ ]:
result_df

,Mean Accuracy,Mean F1-Score,Mean Recall,Mean Precision,Mean AUC,Mean Fall-Out,Mean Miss-Rate
0,78,80,90,73,0.78,0,0


In [ ]:
df.mean()

Mean Accuracy      0.808
Mean F1-Score      0.806
Mean Recall        0.802
Mean Precision     0.812
Mean AUC           0.810
Mean Fall-Out      0.038
Mean Miss-Rate     0.040
tersh             70.000
dtype: float64

precision    recall  f1-score   support

         0.0       0.84      0.79      0.81       100
         1.0       0.80      0.85      0.83       100

    accuracy                           0.82       200
   macro avg       0.82      0.82      0.82       200
weighted avg       0.82      0.82      0.82       200


In [86]:
## Create a dataframe from the dictionary
#df = pd.DataFrame([result])

#df.to_csv('result.csv')
df = pd.read_csv("/content/drive/MyDrive/Result/New-Weight/resultB.csv")
df.drop(["Unnamed: 0"] ,axis = 1 ,inplace= True)
result = {
    'Mean Accuracy': 82,
    'Mean F1-Score': 83,
    'Mean Recall': 85,
    'Mean Precision': 80,
    'Mean AUC': round(roc_auc_score(labels, inp.squeeze()), 2),
    'Mean Fall-Out':0,
    'Mean Miss-Rate':0,
    "tresh" : 50
}

result_df = pd.DataFrame([result])

# Concatenate the two DataFrames vertically
df = pd.concat([df, result_df], ignore_index=True)
df.to_csv('/content/drive/MyDrive/Result/New-Weight/resultB.csv')

df

,Mean Accuracy,Mean F1-Score,Mean Recall,Mean Precision,Mean AUC,Mean Fall-Out,Mean Miss-Rate,tresh
0,80,81,88,76,0.80,0,0,55
1,81,82,87,78,0.82,0,0,50
2,80,81,83,78,0.80,0,0,55
3,80,80,80,80,0.80,0,0,50
4,82,83,85,80,0.82,0,0,50


In [87]:
df.mean()

Mean Accuracy     80.600
Mean F1-Score     81.400
Mean Recall       84.600
Mean Precision    78.400
Mean AUC           0.808
Mean Fall-Out      0.000
Mean Miss-Rate     0.000
tresh             52.000
dtype: float64